In [4]:
import numpy as np
import time
import datetime
import torch
import torch.nn as nn

In [2]:
def quant_cross(y_pred):
        if len(np.shape(y_pred)) == 3:
            crosscount = 0
            for i in range(np.shape(y_pred)[0]):
                for j in range(np.shape(y_pred)[1]):
                    for k in range(np.shape(y_pred)[2]-1):
                        if y_pred[i,j,k+1] < y_pred[i,j,k]:
                            crosscount = crosscount + 1 

        else:
            crosscount = 0
            for i in range(np.shape(y_pred)[0]):
                for j in range(np.shape(y_pred)[1]-1):
                    if y_pred[i,j+1] < y_pred[i,j]:
                        crosscount = crosscount + 1 

        crossrate = crosscount/(np.size(y_pred)-np.size(y_pred[...,0]))

        return crossrate


In [3]:
quants = np.array([[1,2,3],[3,2,1]])

quant_cross(quants)

0.5

In [4]:
quantiles=np.array([0.1,0.5,0.9])
est= np.where(quantiles==0.5)[0].item()
print(est)

1


In [4]:
datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H-%M-%S')

'2024-03-11_13-13-15'

In [5]:
RMSE_train = np.load('log/2024-03-11_13-47-44/train_stats/RMSE_train.npz', allow_pickle=True)

In [11]:
RMSE_train

NpzFile 'log/2024-03-11_13-47-44/train_stats/RMSE_train.npz' with keys: values, means, mas, times

In [12]:

means = RMSE_train['mas']
np.size(means)
means

array([3.53125024, 3.65625012, 4.15625008, ..., 4.48533434, 4.48463026,
       4.48589563])

In [1]:
A=[0,1,2,3,4,5,6,7,8,9,10,11,12,13]
A[:12]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [4]:
A=np.array([500,500,500,500])
B=np.array([0,0,2000,0])

print(np.mean(np.abs(A)))
print(np.mean(np.abs(B)))
print(np.sqrt(np.mean(A**2)))
print(np.sqrt(np.mean(B**2)))

500.0
500.0
500.0
1000.0


In [7]:
#Set up NN structure
no_nodes = 100

sequence= nn.Sequential(
    nn.Linear(13,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.Linear(no_nodes, 3*12) #Output dimesion is number of quantiles times number of target variables
)

print(sequence)

Sequential(
  (0): Linear(in_features=13, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=100, bias=True)
  (5): ReLU()
  (6): Linear(in_features=100, out_features=100, bias=True)
  (7): ReLU()
  (8): Linear(in_features=100, out_features=100, bias=True)
  (9): ReLU()
  (10): Linear(in_features=100, out_features=100, bias=True)
  (11): ReLU()
  (12): Linear(in_features=100, out_features=36, bias=True)
)


In [8]:
no_layers=5
modules = []
modules.append(nn.Linear(len(X_cols),no_nodes))
modules.append(nn.ReLU())
for i in range(no_layers):
    modules.append(nn.Linear(no_nodes,no_nodes))
    modules.append(nn.ReLU())
modules.append(nn.Linear(no_nodes, len(quantiles)*len(y_cols)))
sequence = lambda: nn.Sequential(*modules)

print(sequential)

Sequential(
  (0): Linear(in_features=13, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=100, bias=True)
  (5): ReLU()
  (6): Linear(in_features=100, out_features=100, bias=True)
  (7): ReLU()
  (8): Linear(in_features=100, out_features=100, bias=True)
  (9): ReLU()
  (10): Linear(in_features=100, out_features=100, bias=True)
  (11): ReLU()
  (12): Linear(in_features=100, out_features=36, bias=True)
)
